# Transfer scrap from LinkedIn to Airtable

Takes a JSON file that was scrapped with data from LinkedIn and makes it ready to import to Airtable

In [2]:
import pandas
import airtable
import os

In [3]:
PATH = ''
df = pandas.read_json(PATH, orient='records', encoding='utf-8')


FileNotFoundError: File Z:/- RESPONSABLE DIGITAL/SCRIPTS/siec_linkedin.json does not exist

In [29]:
df['enriched'] = 'Enrichi'

## Airtable
Convert the initial json format to the Airtable fields

In [30]:
instance = airtable.Airtable(
    os.getenv('AIRTABLE_BASE_ID'),
    os.getenv('SCRAPING_TABLE_NAME'), 
    os.getenv('AIRTABLE_API_KEY')
)

In [31]:
CONVERSION_COLUMNS = {
    'company': 'Entreprise',
    'linkedin': 'LinkedIn',
    'full_name': 'Nom complet',
    'company_linkedin': 'Company LinkedIn',
    'enriched': 'Statut enrichissement',
    'website': 'Site entreprise',
    'first_name': 'Prénom',
    'last_name': 'Nom',
    'position': 'Poste'
}
airtable_df = df.rename(columns=CONVERSION_COLUMNS)

In [32]:
data = airtable_df.to_json(orient='records', force_ascii=False)

In [33]:
# instance.batch_insert(data)

In [34]:
df.to_csv('data/linkedin_profiles.csv', index=False)